In [1]:
'''from pyspark.sql import SparkSession
from delta import configure_spark_with_delta_pip'''

'from pyspark.sql import SparkSession\nfrom delta import configure_spark_with_delta_pip'

In [2]:
'''import pyspark
from delta import *

builder = pyspark.sql.SparkSession.builder.appName("MyApp") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config('spark.jars.packages', 'org.apache.spark:spark-sql-kafka-0-10_2.12:3.3.0') \
    .config("spark.sql.shuffle.partitions", 4) \

spark = configure_spark_with_delta_pip(builder).getOrCreate()'''

'import pyspark\nfrom delta import *\n\nbuilder = pyspark.sql.SparkSession.builder.appName("MyApp")     .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")     .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")     .config(\'spark.jars.packages\', \'org.apache.spark:spark-sql-kafka-0-10_2.12:3.3.0\')     .config("spark.sql.shuffle.partitions", 4) \nspark = configure_spark_with_delta_pip(builder).getOrCreate()'

In [3]:
'''print(spark.sparkContext.getConf().get("spark.driver.extraClassPath"))
print(spark.sparkContext.getConf().get("spark.executor.extraClassPath"))
'''

'print(spark.sparkContext.getConf().get("spark.driver.extraClassPath"))\nprint(spark.sparkContext.getConf().get("spark.executor.extraClassPath"))\n'

In [4]:
'''df = (
    spark.readStream.format("kafka")
    .option("kafka.bootstrap.servers", "kafka:29092")
    .option("subscribe", "users_created")
    .option("startingOffsets", "earliest") 
    .load()
)
'''

'df = (\n    spark.readStream.format("kafka")\n    .option("kafka.bootstrap.servers", "kafka:29092")\n    .option("subscribe", "users_created")\n    .option("startingOffsets", "earliest") \n    .load()\n)\n'

In [5]:
# Create the Spark Session
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Streaming from Kafka") \
    .config("spark.streaming.stopGracefullyOnShutdown", True) \
    .config('spark.jars.packages', 'org.apache.spark:spark-sql-kafka-0-10_2.12:3.3.0') \
    .config("spark.sql.shuffle.partitions", 4) \
    .master("local[*]") \
    .getOrCreate()

spark

:: loading settings :: url = jar:file:/usr/local/Homebrew/Caskroom/miniforge/base/envs/data_pipeline/lib/python3.9/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/hamzaharunamohammed/.ivy2/cache
The jars for the packages stored in: /Users/hamzaharunamohammed/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-c6bee2e0-240c-414d-b9c4-5b2c1ed2bfe1;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.3.0 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.3.0 in central
	found org.apache.kafka#kafka-clients;2.8.1 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.8.4 in central
	found org.slf4j#slf4j-api;1.7.32 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.2 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.hadoop#hadoop-client-api;3.3.2 in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found org.apache.commons#commons-poo

In [6]:
# Create the streaming_df to read from kafka
streaming_df = spark.readStream\
    .format("kafka") \
    .option("kafka.bootstrap.servers", "kafka:29092") \
    .option("subscribe", "users_created") \
    .option("startingOffsets", "earliest") \
    .load()

In [7]:
streaming_df

DataFrame[key: binary, value: binary, topic: string, partition: int, offset: bigint, timestamp: timestamp, timestampType: int]

In [8]:
from pyspark.sql.types import StructField, StringType, IntegerType, StructType
from pyspark.sql.functions import from_json, col

schema = StructType([
        StructField("id", IntegerType(), False),
        StructField("first_name", StringType(), False),
        StructField("last_name", StringType(), False),
        StructField("gender", StringType(), False),
        StructField("address", StringType(), False),
        StructField("postcode", IntegerType(), False),
        StructField("email", StringType(), False),
        StructField("username", StringType(), False),
        StructField("dob", StringType(), False),
        StructField("registered", StringType(), False),
        StructField("phone", StringType(), False),
        StructField("picture", StringType(), False)
    ])

df = streaming_df.selectExpr("CAST(value AS STRING)").select(from_json(col("value"), schema).alias("data")).select(
        "data.*")

df

DataFrame[id: int, first_name: string, last_name: string, gender: string, address: string, postcode: int, email: string, username: string, dob: string, registered: string, phone: string, picture: string]

In [9]:
type(streaming_df)

pyspark.sql.dataframe.DataFrame

In [12]:
# view streaming_df and df

df.writeStream.format("console").start().awaitTermination()

24/08/15 19:02:35 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /private/var/folders/qb/fvvyb5ks2nd3hqyvm2stxxjh0000gn/T/temporary-6e3cb158-7be8-400f-bff8-ce0540a3682d. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/08/15 19:02:35 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
24/08/15 19:02:35 WARN ClientUtils: Couldn't resolve server kafka:29092 from bootstrap.servers as DNS resolution failed for kafka
24/08/15 19:02:35 WARN KafkaOffsetReaderAdmin: Error in attempt 1 getting Kafka offsets: 
org.apache.kafka.common.KafkaException: Failed to create new KafkaAdminClient
	at org.apache.kafka.clients.admin.KafkaAdminClient.createInternal(KafkaAdminClient.java:540)
	at org.apache.kafka.client

StreamingQueryException: [STREAM_FAILED] Query [id = 6c8bd21e-f06a-47fb-b436-837402735dba, runId = ef35c6c9-d41a-4d95-9e7e-7068fe4b342f] terminated with exception: Failed to create new KafkaAdminClient

24/08/17 16:05:35 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 1082465 ms exceeds timeout 120000 ms
24/08/17 16:05:35 WARN SparkContext: Killing executors is not supported by current scheduler.
24/08/17 16:05:44 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:124)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$